In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../data/MEX/")[0:200]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

200 municipalities.


In [4]:
image_names[0].split("/")[3]

'484001008'

In [5]:
# image_names

In [6]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .001)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .60,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [8]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [9]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [10]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  120
Num validation:  80


In [11]:
# for i,o in train_dl:
#     print(i)
# #     print(load_inputs(i[0]).shape, o)

In [12]:
butler.train_attn_model(train_dl, val_dl)

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch:  0
  Training Loss:  4317.459443505605
  Validation Loss:  5081.584721136093


Epoch:  1
  Training Loss:  4257.111666297913
  Validation Loss:  5034.443544483185


Epoch:  2
  Training Loss:  4213.97881380717
  Validation Loss:  5013.788635921478


Epoch:  3
  Training Loss:  4186.198096084595
  Validation Loss:  5025.30317993164


Epoch:  4
  Training Loss:  4167.8922757466635
  Validation Loss:  4924.080422019959
VALS BEFORE:  [[-316.87048112112825, 0, 100.0], [0, 0, 100.0], [0, 100.0, -30785.526315789473], [100.0, -17.825258639400005, -1138.1165651498866], [0, 0, 0], [0, 0, 100.0], [0, 0, 0], [0, 100.0, 0], [0, 100.0, 51.84237116874368], [0, 100.0, 0], [0, 0, 0], [0, 100.0, 0], [0, 0, 0], [8.64965774735532, 0, 100.0], [100.0, -3.232457819485182, -1223.686205658194], [100.0, -41.01211014139774, 0], [0, 0, 100.0], [100.0, -11455.802807257789, 78.54572163055454], [100.0, -61.67952727784331, -13810.575837410614], [0, 0, 0], [0, 0, 0], [100.0, 0.0, 0.0], [0, 0, 0], [0, 0, 100.0],

KeyboardInterrupt: 

In [ ]:
butler.image_sizes